In [1]:
import os
import sys

import numpy as np

run_env = os.getenv('RUN_ENV', 'COLLAB')
if run_env == 'COLLAB':
  from google.colab import drive
  ROOT_DIR = '/content/drive'
  drive.mount(ROOT_DIR)
  print('Google drive connected')
  root_data_dir = os.path.join(ROOT_DIR, 'MyDrive', 'ml_course_data')
  lib_path = os.path.join(ROOT_DIR, 'MyDrive', 'src')
  if not os.path.exists(lib_path):
    raise RuntimeError('Upload and `src` dir with code')
  sys.path.append(lib_path)
else:
  root_data_dir = os.getenv('DATA_DIR', '/srv/data')

if not os.path.exists(root_data_dir):
  raise RuntimeError('Data dir not exists')
else:
  print('Data dir content %s: %s' % (root_data_dir, ', '.join(os.listdir(root_data_dir)[:5])))

Data dir content /Users/adzhumurat/PycharmProjects/ai_product_engineer/data: client_segmentation.csv, messages.db, labeled_data_corpus.csv, chroma, content_description.csv


In [2]:
from rag.connections import get_chroma_client

client = get_chroma_client()
client

Connecting to ChromaDB service at http://localhost:8000


In [20]:
import requests

host = 'http://0.0.0.0:11434'
query = 'что такое антиградиент'
timeout = 5

model = 'granite4:350m'
model = 'qwen2.5:1.5b'

url = host.rstrip("/") + "/api/embeddings"
response = requests.post(
    url,
    json={"model": model, "prompt": query},
    timeout=timeout,
)
response.raise_for_status()
data = response.json()

query_embedding = data.get("embedding") or data.get("embeddings")
print(np.array(query_embedding).shape)

(1536,)


In [21]:
include = ["documents", "metadatas", "distances"]
collection = client.get_collection('docs1')
limit = 10

collection.query(
    query_embeddings=[query_embedding],
    n_results=limit,
    include=include,
)

{'ids': [['44910ac67cd260627cc10b7a1e98ffc0_5',
   '521cd333018c5db9ad1eaf97b0c7b5dd_57',
   'e2a544eb5aafdaa54e2eece40381c0ef_27',
   '521cd333018c5db9ad1eaf97b0c7b5dd_49',
   'e102ed911c56b270ee42a7663c35e4b8_1',
   '521cd333018c5db9ad1eaf97b0c7b5dd_39',
   '11ca17fdd8bb5b27217f9691effa4e03_5',
   '87e5ac00900a7395167b7338d840887e_6',
   '44afae845aa8ae52cd29d3ec7e50f745_14',
   'e2a544eb5aafdaa54e2eece40381c0ef_39']],
 'distances': [[5993.615,
   6101.614,
   6239.2065,
   6521.266,
   6534.8027,
   6565.432,
   6649.8496,
   6649.8496,
   6664.7915,
   6682.0903]],
 'embeddings': None,
 'metadatas': [[{'source_file_name': 'vol_04_deep_dive_09_trees_boosting.ipynb',
    'chunk_index': '5',
    'source': 'data/md_docs/vol_04_deep_dive_09_trees_boosting.md',
    'source_dir': '/Users/adzhumurat/PycharmProjects/ai_product_engineer/jupyter_notebooks'},
   {'chunk_index': '57',
    'source_file_name': 'vol_04_deep_dive_00_probability.ipynb',
    'source': 'data/md_docs/vol_04_deep_dive_0

In [25]:
docs_path = os.path.join(root_data_dir, 'md_docs')
for current_file in os.listdir(docs_path):
    file_path = os.path.join(docs_path, current_file)
    with open(file_path, 'r') as f:
        file_content = f.read()
    print(len(file_content))

25743
12328
50755
22446
13410
12348
23547
9653
45966
22105
19253
15822
26619
9263
13511
2108
32743
31300
25222
40373
6008
42048
35539
12521
8400
50919
2246
13204
15410
1065
14506
31357
20802
23344
35824
21435
